<a href="https://colab.research.google.com/github/TheForeverLost/FootballTransfers/blob/master/transfer_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Collection for transfer analysis


## Imports
We will be using bs4 to parse through the html pages

In [0]:
from bs4 import BeautifulSoup
import urllib.request
import re

import pandas as pd
import numpy as np

from urllib.request import Request, urlopen
from random import randint,shuffle
import time
from time import sleep


In [0]:
!pip install user_agent
import user_agent
from user_agent import generate_user_agent, generate_navigator

## Scraping News Websites for traffic news

### Skysports
Skysports does not hold old transfer news (only 12 pages of it) on their website but they are lenient when it comes to scrappers

In [0]:
content = []
skysports = [
       "https://www.skysports.com/football/transfer-news",
       "https://www.skysports.com/football/transfer-news/more/2",
       "https://www.skysports.com/football/transfer-news/more/3",
       "https://www.skysports.com/football/transfer-news/more/4",
       "https://www.skysports.com/football/transfer-news/more/5",
       "https://www.skysports.com/football/transfer-news/more/6",
       "https://www.skysports.com/football/transfer-news/more/7",
       "https://www.skysports.com/football/transfer-news/more/8",
       "https://www.skysports.com/football/transfer-news/more/9",
       "https://www.skysports.com/football/transfer-news/more/10",
       "https://www.skysports.com/football/transfer-news/more/11",
       "https://www.skysports.com/football/transfer-news/more/12"
       ]
for url in skysports:
    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page, 'html.parser')
    regex = re.compile('news-item')
    content_lis = soup.find_all('div', attrs={'class': regex})
    for div in content_lis:
        title = div.find('h2').getText()
        link = div.find('a')["href"]
        x = re.search("^/transfer" , link)
        if x != None :
          link = "https://www.skysports.com" + link;
        topic = div.find('strong').getText().strip()
        newspage = urllib.request.urlopen(link)
        tsoup = BeautifulSoup(newspage, 'html.parser')
        try:
          article_title = tsoup.find('h1').getText()
        except:
          article_title = ""
        try:
          article_subtitle = tsoup.find('h2').getText()
        except:
          article_subtitle = ""
        h3_array = tsoup.find_all('h3')
        article_part = []
        for h3 in h3_array : 
            article_part.append(h3.getText())
        article_part = '\n'.join(article_part)
        p_array = tsoup.find_all('p')
        article_content = []
        for h3 in p_array : 
            article_content.append(h3.getText())
        article_content = '\n'.join(article_content)
        regex = re.compile("highlight")
        try:
          date = tsoup.find("span" , attrs={"class":regex}).getText()
        except:
          date = ""
        article = {
            "url" : link ,
            "subscript" : title ,
            "topic" : topic ,
            "title" : article_title ,
            "subtitle" : article_subtitle ,
            "emphasis" : article_part ,
            "content" : article_content ,
            "date" : date
        }
        content.append(article)
df = pd.DataFrame(content)
df.to_csv("skysport_data.csv")

### Goal
Goal.com has all its articles accessible but they have a strict robots.txt so we'll have to use a user agent to scrape data
We'll also put a time delay as we are scraping a lot of information we do not want overload their servers with requests.

* **Irresponsible web scraping can get you blocked from websites** 
* **Data scraped belongs to goal.com. Any form of redistribution will have legal consequences**

In [0]:
headers = {'User-Agent': generate_user_agent()}
content = []
links = []
goal = [
        "https://www.goal.com/en-us/transfer-news"
       ]
for i in range(2,501):
  new_url = goal[0]+"/"+str(i)
  goal.append(new_url)
shuffle(goal)

In [0]:
import time
n = 0
program_starts = time.time()
for url in goal:
  print(url)
  now = time.time()
  print("{0} sites have been scraped .It has been {1} seconds".format(n,now - program_starts))
  sleep(randint(1,30))
  request = Request(url, headers=headers)
  page = urllib.request.urlopen(request)
  soup = BeautifulSoup(page, 'html.parser')
  content_lis = soup.find_all('article')
  for article in content_lis :
    regex = re.compile("article")
    link = article.find('a', attrs={'class': regex})
    if link != None:
      link = "https://www.goal.com"+link["href"]  
      links.append(link)
  n = n + 1
links = pd.DataFrame(links)
links.to_csv("goal.csv")  

In [0]:
df = pd.DataFrame()
links = pd.read_csv("goal.csv")
links = list(links["0"])

In [0]:
shuffle(links)

In [0]:
from tqdm import tqdm
headers = {'User-Agent': generate_user_agent()}
n = 0
program_starts = time.time()
for url in tqdm(links):
  try:
    n = n+1
    request = Request(url, headers=headers)
    page = urllib.request.urlopen(request)
    soup = BeautifulSoup(page, 'html.parser')
    try:
      article_title = soup.find("h1").getText()
    except:
      article_title = ""
    try:
      regex = re.compile("teaser")
      article_subtitle = soup.find("div",attrs={"class" : regex}).getText()
    except:
      article_subtitle = ""
    h3_array = soup.find_all('h2')
    article_part = []
    for h3 in h3_array : 
        article_part.append(h3.getText())
    article_part = '\n'.join(article_part)
    p_array = soup.find_all('p')
    article_content = []
    for h3 in p_array : 
        article_content.append(h3.getText())
    article_content = '\n'.join(article_content)
    regex = re.compile("tags-list__link")
    taglist = soup.find_all("a",attrs={"class" : regex})
    tags = []
    for tag in taglist:
      tags.append(tag.getText())
    topic = '+'.join(tags)
    datetag = soup.find("time")
    date = datetag.getText()
    article = {
              "url" : url ,
              "subscript" : article_title ,
              "topic" : topic ,
              "title" : article_title ,
              "subtitle" : article_subtitle ,
              "emphasis" : article_part ,
              "content" : article_content ,
              "date" : date.strip()
          }
    df = df.append(article , ignore_index=True)  
    df.to_csv("transfer_data.csv")
  except:
    n=n-1

100%|██████████| 13013/13013 [5:48:17<00:00,  1.61s/it]


In [0]:
df.shape

(12875, 8)

# Analysis of transfer reports


## Training name entity recognition

In [0]:
from google.colab import drive
drive.mount("/content/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pandas as pd
import spacy
from spacy import displacy
import numpy

In [0]:
df = pd.read_csv("/content/gdrive/My Drive/project stuff/transfer_data.csv")

In [0]:
df

,content,date,emphasis,subscript,subtitle,title,topic,url
0,Kane suffered a serious set-back on New Year'...,1/9/20,Trending News,Dembele on Tottenham's radar following Kane su...,News that the England striker will miss at le...,Dembele on Tottenham's radar following Kane su...,Transfers + Tottenham Hotspur + Olympique Lyo...,https://www.goal.com/en-us/news/dembele-on-tot...
1,Cristian Tello will leave Barcelona to join R...,6/30/17,Trending News,Tello to join Real Betis from Barcelona in €5m...,"After re-signing Gerard Deulofeu, the Catalan...",Tello to join Real Betis from Barcelona in €5m...,Transfers + Primera División + Tello + Real B...,https://www.goal.com/en-us/news/cristian-tello...
2,Pep Guardiola accepts Nicolas Otamendi is not...,1/22/19,Trending News,'I don't want sad players here' - Guardiola ad...,Nicolas Otamendi has drifted out of favour at...,'I don't want sad players here' - Guardiola ad...,Transfers + Manchester City + Nicolás Otamend...,https://www.goal.com/en-us/news/i-dont-want-sa...
3,Paul Clement has warned in-demand Gylfi Sigur...,5/19/17,Trending News,Sigurdsson should remember Tottenham disappoin...,The Iceland international was unable to secur...,Sigurdsson should remember Tottenham disappoin...,Transfers + Swansea City + Tottenham Hotspur ...,https://www.goal.com/en-us/news/premier-league...
4,Neymar’s father has refuted reports suggestin...,7/2/19,Trending News,Neymar's father offers update on transfer plan...,Reports suggesting that discussions with Camp...,Neymar's father offers update on transfer plan...,Transfers + Barcelona + Neymar + Primera Divi...,https://www.goal.com/en-us/news/neymars-father...
...,...,...,...,...,...,...,...,...
12870,Antoine Griezmann understands why Atletico Ma...,6/19/18,Trending News\nRelated,Griezmann understands Atletico fans' nerves ov...,After Barcelona were linked with signing the ...,Griezmann understands Atletico fans' nerves ov...,Transfers + Atlético Madrid + Atlético Madrid...,https://www.goal.com/en-us/news/griezmann-unde...
12871,Victor Lindelof has spoken of the continued s...,4/4/17,Trending News,Lindelof breaks silence over Man Utd links,The defender has long been linked with a move...,Lindelof breaks silence over Man Utd links,Benfica + Manchester United + Premier League ...,https://www.goal.com/en-us/news/lindelof-break...
12872,Sydney FC are believed to be interested in si...,12/5/17,Trending News\nRelated,The Covert Agent: Sydney FC interested in snar...,EXCLUSIVE: Goal's resident spy reveals Graham...,The Covert Agent: Sydney FC interested in snar...,Transfers + A-League + Sydney + Nick Fitzgera...,https://www.goal.com/en-us/news/the-covert-age...
12873,Luka Jovic said his dream has come true after...,6/6/19,Trending News,'My dream comes true now' - Madrid recruit Jov...,The forward said his goodbyes to the German s...,'My dream comes true now' - Madrid recruit Jov...,Transfers + Real Madrid + Luka Jović + Primer...,https://www.goal.com/en-us/news/my-dream-comes...


In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
!python -m spacy download en_core_web_md

     |████████████████████████████████| 96.4MB 1.1MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-cp36-none-any.whl size=98051305 sha256=9ca508bde0a3e28f0e4d4316108ba34b0e7dd2852329748a0dfd68ca812c4597
  Stored in directory: /tmp/pip-ephem-wheel-cache-xgrktl1f/wheels/df/94/ad/f5cf59224cea6b5686ac4fd1ad19c8a07bc026e13c36502d81
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [0]:
import en_core_web_md

In [0]:
nlp = en_core_web_md.load()
doc = nlp(u"Kylian Mbappe to force a move from Paris Saint Germain to join Real Madrid in 2021")

In [0]:
for token in doc.ents :
  print(token.text,spacy.explain(token.label_))

Kylian Mbappe People, including fictional
Paris Saint Germain Buildings, airports, highways, bridges, etc.
Real Madrid Companies, agencies, institutions, etc.
2021 Absolute or relative dates or periods


In [0]:
from bs4 import BeautifulSoup
import urllib.request
from urllib.request import Request, urlopen
import re

In [0]:
!pip install user_agent
import user_agent
from user_agent import generate_user_agent, generate_navigator

  Created wheel for user-agent: filename=user_agent-0.1.9-cp36-none-any.whl size=18807 sha256=b444b63614071842f423ca822d27d034bc6922592a1ea3d4f900f35caf3aa091
  Stored in directory: /root/.cache/pip/wheels/92/80/3f/5d79277825042f2d4d447f594e3fc046f1e506f2b481d364b2
Successfully built user-agent


In [0]:
headers = {'User-Agent': generate_user_agent()}
url = "https://www.uefa.com/memberassociations/leaguesandcups/index.html" 
request = Request(url, headers=headers)
page = urllib.request.urlopen(request)
soup = BeautifulSoup(page, 'html.parser')
regex = re.compile('^\d+')
countries = [(x.text,"https://www.uefa.com/"+x.find("a")["href"]) for x in soup.find_all('li', attrs={'class': regex})]

In [0]:
teams = []
for country,url in countries:
  request = Request(url, headers=headers)
  page = urllib.request.urlopen(request)
  soup = BeautifulSoup(page, 'html.parser')
  regex = re.compile('^club')
  teams = teams + [(team.text , "https://www.uefa.com/"+team.find("a")["href"],country) for team in soup.find_all("td" , attrs={"class",regex})]

In [0]:
from tqdm import tqdm
players = []
for team in tqdm(teams):
  url = team[1]
  request = Request(url, headers=headers)
  page = urllib.request.urlopen(request)
  soup = BeautifulSoup(page, 'html.parser')
  regex = re.compile('^playername')
  players = players + [(x.text,team[0]) for x in soup.find_all("td",attrs={"class":regex})]

100%|██████████| 721/721 [13:56<00:00,  1.16s/it]


In [0]:
from random import shuffle
shuffle(teams) 
shuffle(players)

In [0]:
TRAINING_DATA = []

### Training the named entity recognizer of a en_core_web_md

In [0]:
LABEL1 = "TEAM"
LABEL2 = "PLAYER"

In [0]:
for team in teams[:200]:
  sentence = team[0]+" is a club from "+team[2]+"."
  ent = {"entities" : [(0,len(team[0]),LABEL1),(len(team[0])+16,len(sentence)-1,"GPE")]}
  TRAINING_DATA = TRAINING_DATA + [(sentence , ent)]
print(TRAINING_DATA[-1])

for team in teams[200:400]:
  sentence = team[0]+" plays in "+team[2]+"."
  ent = {"entities" : [(0,len(team[0]),LABEL1),(len(team[0])+10,len(sentence)-1,"GPE")]}
  TRAINING_DATA = TRAINING_DATA + [(sentence , ent)]
print(TRAINING_DATA[-1])

for team in teams[400:550]:
  sentence = team[0]+" is a successful club."
  ent = {"entities" : [(0,len(team[0]),LABEL1)]}
  TRAINING_DATA = TRAINING_DATA + [(sentence , ent)]
print(TRAINING_DATA[-1])

for team in teams[550:]:
  sentence = "The "+team[2]+" based club "+team[0]+"."
  ent = {"entities" : [(4,len(team[2]),"GPE"),(16+len(team[2]),16+len(team[2])+len(team[0]),LABEL1)]}
  TRAINING_DATA = TRAINING_DATA + [(sentence , ent)]
print(TRAINING_DATA[-1])

for player in players[0:1000]:
  sentence = player[0]+" wants to join "+player[1]
  ent = {
      "entities" : [
                    (0 , len(player[0]) ,LABEL2 ) ,
                    (15 + len(player[0]), 15 + len(player[0]) + len(player[1]) , LABEL1)
                    ]
      }
  TRAINING_DATA = TRAINING_DATA + [(sentence , ent)]
print(TRAINING_DATA[-1])

for player in players[1500:3000]:
  sentence = player[0]+" is looking to leave "+player[1]
  ent = {
      "entities" : [
                    (0 , len(player[0]) ,LABEL2 ) ,
                    (21 + len(player[0]), 21 + len(player[0]) + len(player[1]) , LABEL1)
                    ]
      }
  TRAINING_DATA = TRAINING_DATA + [(sentence , ent)]
print(TRAINING_DATA[-1])

for player in players[3000:4500]:
  sentence = player[0]+" is happy at "+player[1]
  ent = {
      "entities" : [
                    (0 , len(player[0]) ,LABEL2 ) ,
                    (13 + len(player[0]), 13 + len(player[0]) + len(player[1]) , LABEL1)
                    ]
      }
  TRAINING_DATA = TRAINING_DATA + [(sentence , ent)]
print(TRAINING_DATA[-1])

for player in players[4500:6000]:
  sentence = player[1]+" wants to sign "+player[0]
  ent = {
      "entities" : [
                    (0 , len(player[1]) ,LABEL1 ) ,
                    (15 + len(player[1]), 15 + len(player[0]) + len(player[1]) , LABEL2)
                    ]
      }
  TRAINING_DATA = TRAINING_DATA + [(sentence , ent)]
print(TRAINING_DATA[-1])

for player in players[6000:7500]:
  sentence = player[1]+" makes offer for "+player[0]
  ent = {
      "entities" : [
                    (0 , len(player[1]) ,LABEL1 ) ,
                    (16 + len(player[1]), 16 + len(player[0]) + len(player[1]) , LABEL2)
                    ]
      }
  TRAINING_DATA = TRAINING_DATA + [(sentence , ent)]
print(TRAINING_DATA[-1])

for player in players[7500:]:
  sentence = player[1]+" trying to kick out "+player[0]
  ent = {
      "entities" : [
                    (0 , len(player[1]) ,LABEL1 ) ,
                    (20 + len(player[1]), 20 + len(player[0]) + len(player[1]) , LABEL2)
                    ]
      }
  TRAINING_DATA = TRAINING_DATA + [(sentence , ent)]
print(TRAINING_DATA[-1])



('Vushtrria is a club from Kosovo.', {'entities': [(0, 9, 'TEAM'), (25, 31, 'GPE')]})
('FC Rubin plays in Russia.', {'entities': [(0, 8, 'TEAM'), (18, 24, 'GPE')]})
('KF Shkëndija is a successful club.', {'entities': [(0, 12, 'TEAM')]})
('The Kazakhstan based club FC Zhetysu Taldykorgan.', {'entities': [(4, 10, 'GPE'), (26, 48, 'TEAM')]})
('Valentin Gendrey wants to join Amiens', {'entities': [(0, 16, 'PLAYER'), (31, 37, 'TEAM')]})
('David Pavelka is looking to leave Kasımpaşa SK', {'entities': [(0, 13, 'PLAYER'), (34, 46, 'TEAM')]})
('Chem Campbell is happy at Wolverhampton Wanderers FC', {'entities': [(0, 13, 'PLAYER'), (26, 52, 'TEAM')]})
('FC Metz wants to sign Farid Boulaya', {'entities': [(0, 7, 'TEAM'), (22, 35, 'PLAYER')]})
('FC St. Gallen makes offer for Fabiano Donato Alves', {'entities': [(0, 13, 'TEAM'), (29, 49, 'PLAYER')]})
('RCD Mallorca trying to kick out Ante Budimir', {'entities': [(0, 12, 'TEAM'), (32, 44, 'PLAYER')]})


In [0]:
len(TRAINING_DATA)

9207

In [0]:
ner = nlp.get_pipe("ner")
ner.add_label(LABEL1)
ner.add_label(LABEL2)

In [0]:
move_names = list(ner.move_names)
print(move_names)
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
other_pipes

['B-ORG', 'B-DATE', 'B-PERSON', 'B-GPE', 'B-MONEY', 'B-CARDINAL', 'B-NORP', 'B-PERCENT', 'B-WORK_OF_ART', 'B-LOC', 'B-TIME', 'B-QUANTITY', 'B-FAC', 'B-EVENT', 'B-ORDINAL', 'B-PRODUCT', 'B-LAW', 'B-LANGUAGE', 'I-ORG', 'I-DATE', 'I-PERSON', 'I-GPE', 'I-MONEY', 'I-CARDINAL', 'I-NORP', 'I-PERCENT', 'I-WORK_OF_ART', 'I-LOC', 'I-TIME', 'I-QUANTITY', 'I-FAC', 'I-EVENT', 'I-ORDINAL', 'I-PRODUCT', 'I-LAW', 'I-LANGUAGE', 'L-ORG', 'L-DATE', 'L-PERSON', 'L-GPE', 'L-MONEY', 'L-CARDINAL', 'L-NORP', 'L-PERCENT', 'L-WORK_OF_ART', 'L-LOC', 'L-TIME', 'L-QUANTITY', 'L-FAC', 'L-EVENT', 'L-ORDINAL', 'L-PRODUCT', 'L-LAW', 'L-LANGUAGE', 'U-ORG', 'U-DATE', 'U-PERSON', 'U-GPE', 'U-MONEY', 'U-CARDINAL', 'U-NORP', 'U-PERCENT', 'U-WORK_OF_ART', 'U-LOC', 'U-TIME', 'U-QUANTITY', 'U-FAC', 'U-EVENT', 'U-ORDINAL', 'U-PRODUCT', 'U-LAW', 'U-LANGUAGE', 'O', 'B-TEAM', 'I-TEAM', 'L-TEAM', 'U-TEAM', 'B-PLAYER', 'I-PLAYER', 'L-PLAYER', 'U-PLAYER']


['tagger', 'parser']

In [0]:
from spacy.util import compounding , minibatch
n_iter = 50
optimizer = nlp.resume_training()
with nlp.disable_pipes(*other_pipes):  # only train NER
  sizes = compounding(1.0, 4.0, 1.001)
  for itn in tqdm(range(n_iter)):
      shuffle(TRAINING_DATA)
      batches = minibatch(TRAINING_DATA, size=sizes)
      losses = {}
      for batch in batches:
          texts, annotations = zip(*batch)
          nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
      print("Losses", losses)

In [0]:
!pip install pathlib
from pathlib import Path
new_model_name = "en_fnew_md"
output_dir = "./content/gdrive/My Drive/project stuff/en_fnew_md"
output_dir = Path(output_dir)
if not output_dir.exists():
    output_dir.mkdir()
nlp.meta["name"] = new_model_name  # rename model
nlp.to_disk(output_dir)
print("Saved model to", output_dir)

Saved model to en_fnew_md


In [0]:
!zip -r fmodel.zip en_fnew_md

updating: en_fnew_md/ (stored 0%)
  adding: en_fnew_md/parser/ (stored 0%)
  adding: en_fnew_md/parser/model (deflated 7%)
  adding: en_fnew_md/parser/moves (deflated 56%)
  adding: en_fnew_md/parser/cfg (deflated 49%)
  adding: en_fnew_md/meta.json (deflated 46%)
  adding: en_fnew_md/tagger/ (stored 0%)
  adding: en_fnew_md/tagger/model (deflated 7%)
  adding: en_fnew_md/tagger/cfg (deflated 34%)
  adding: en_fnew_md/tagger/tag_map (deflated 50%)
  adding: en_fnew_md/ner/ (stored 0%)
  adding: en_fnew_md/ner/model (deflated 9%)
  adding: en_fnew_md/ner/moves (deflated 78%)
  adding: en_fnew_md/ner/cfg (deflated 49%)
  adding: en_fnew_md/tokenizer (deflated 79%)
  adding: en_fnew_md/vocab/ (stored 0%)
  adding: en_fnew_md/vocab/key2row (deflated 78%)
  adding: en_fnew_md/vocab/strings.json (deflated 61%)
  adding: en_fnew_md/vocab/lookups.bin (deflated 45%)
  adding: en_fnew_md/vocab/vectors (deflated 9%)
  adding: en_fnew_md/vocab/lexemes.bin (deflated 64%)


### Completely Training a large language model

We will be training the dependency parser as well as the named entity recognizer again
This time while training the NER we will try making a more natural Training dataset 

#### Training the NER (2nd approach)

In [0]:
LABEL1 = "TEAM"
LABEL2 = "PLAYER"

In [0]:
players[0]

('Karim Rekik', 'Hertha BSC Berlin')

In [0]:
teams[0]

('FK Rad',
 'https://www.uefa.com//teamsandplayers/teams/club=52827/domestic/index.html',
 'Serbia')

In [0]:
templates = [{"sent" : x , "tags" : y} for x,y in zip( df["title"][0:100] , df["topic"][0:100] )]

In [0]:
for tem in templates:
  tags = [x.strip() for x in tem["tags"].split("+")]
  doc = nlp(tem["sent"])
  tem["gaps"] = []
  tem["label"] = []
  print(tem["sent"])
  print(tags)
  for token in doc.ents:
    prev = 0
    if token.text in tags:
      print(token.text , token.label_)
    elif token.label_ in ["PERSON","ORG","GPE"]:
      print(token.text , token.label_)
    ltype = input()
    if ltype == 'p':
      regex = re.compile(token.text)
      loc = re.search(regex , tem["sent"]).span()
      tem["label"].append(LABEL2)
      prev = loc[1]
      tem["sent"] = tem["sent"].replace(token.text , "<PLAYER>" , 1)
    elif ltype == 'o':
      regex = re.compile(token.text)
      loc = re.search(regex , tem["sent"]).span()
      tem["label"].append(LABEL1)
      prev = loc[1]
      tem["sent"] = tem["sent"].replace(token.text , "<TEAM>" , 1)

In [0]:
templates = pd.DataFrame(templates)
templates.to_csv("/content/gdrive/My Drive/project stuff/training_template.csv")

In [0]:
templates = pd.read_csv("/content/gdrive/My Drive/project stuff/training_template.csv")
template = list(templates["sent"])

In [0]:
def getTrainingSample():
  sent = template[randint(0,len(template)-1)]
  ent = {"entities" : [] }
  key = re.search(r"<\w+>" , sent)
  while key != None :
    if key.group() == "<PLAYER>":
      player = players[randint(0,len(players)-1)][0]
      sent = sent.replace("<PLAYER>" , player , 1)
      ent["entities"].append((key.span()[0] , key.span()[0] + len(player) , LABEL2))
    elif key.group() == "<TEAM>":
      team = teams[randint(0,len(teams)-1)][0]
      sent = sent.replace("<TEAM>" , team , 1)
      ent["entities"].append((key.span()[0] , key.span()[0] + len(team) , LABEL1))
    key = re.search(r"<\w+>" , sent)
  return (sent , ent)
print(getTrainingSample())
shuffle(TRAINING_DATA)
TRAINING_DATA = TRAINING_DATA[:2500]

("Klopp defiant on Rasmus Lauritsen's Manchester United FC future: 'I know he will be here'", {'entities': [(17, 33, 'PLAYER'), (36, 56, 'TEAM')]})


In [0]:
for i in range(0,2500):
  TRAINING_DATA.append(getTrainingSample())

In [0]:
import nltk
nltk.download("brown")
from nltk.corpus import brown
for x in brown.fileids()[:10]:
  for y in brown.sents(x):
    TRAINING_DATA.append((
        " ".join(y).replace("`","").replace("'","") ,
        {"entities" : []}
    ))

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [0]:
len(TRAINING_DATA)

6019

In [0]:
nlp = en_core_web_md.load()

In [0]:
list(df["title"])[9983]

"'Leave Real Madrid? I have the same desire to die!' - Navas bullish despite Courtois arrival"

In [0]:
def test_model():
  print("\nTESTING MODEL\n")
  for i in range(0,5):
    text = list(df["title"])[randint(0,10000)]
    doc = nlp(text)
    print("\t"+text)
    print("\t\tEntities", [(ent.text, ent.label_) for ent in doc.ents])
    print("\t\tTokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])
    print("")
test_model()


TESTING MODEL

	Juve risk losing Arsenal and Chelsea target Kean over broken tractor promise
		Entities [('Kean', 'PLAYER')]
		Tokens [('Juve', '', 2), ('risk', '', 2), ('losing', '', 2), ('Arsenal', '', 2), ('and', '', 2), ('Chelsea', '', 2), ('target', '', 2), ('Kean', 'PLAYER', 3), ('over', '', 2), ('broken', '', 2), ('tractor', '', 2), ('promise', '', 2)]

	Seattle Sounders land Peruvian striker Ruidiaz as designated player
		Entities [('Seattle Sounders', 'TEAM'), ('Peruvian', 'PLAYER')]
		Tokens [('Seattle', 'TEAM', 3), ('Sounders', 'TEAM', 1), ('land', '', 2), ('Peruvian', 'PLAYER', 3), ('striker', '', 2), ('Ruidiaz', '', 2), ('as', '', 2), ('designated', '', 2), ('player', '', 2)]

	Hulk reiterates Palmeiras desire but expects Shanghai SIPG stay
		Entities [('Hulk', 'PLAYER'), ('Palmeiras', 'PLAYER'), ('Shanghai SIPG', 'TEAM')]
		Tokens [('Hulk', 'PLAYER', 3), ('reiterates', '', 2), ('Palmeiras', 'PLAYER', 3), ('desire', '', 2), ('but', '', 2), ('expects', '', 2), ('Shanghai',

In [0]:
ner = nlp.get_pipe("ner")
ner.add_label(LABEL1)
ner.add_label(LABEL2)

In [0]:
move_names = list(ner.move_names)
print(move_names)
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
other_pipes

['B-ORG', 'B-DATE', 'B-PERSON', 'B-GPE', 'B-MONEY', 'B-CARDINAL', 'B-NORP', 'B-PERCENT', 'B-WORK_OF_ART', 'B-LOC', 'B-TIME', 'B-QUANTITY', 'B-FAC', 'B-EVENT', 'B-ORDINAL', 'B-PRODUCT', 'B-LAW', 'B-LANGUAGE', 'I-ORG', 'I-DATE', 'I-PERSON', 'I-GPE', 'I-MONEY', 'I-CARDINAL', 'I-NORP', 'I-PERCENT', 'I-WORK_OF_ART', 'I-LOC', 'I-TIME', 'I-QUANTITY', 'I-FAC', 'I-EVENT', 'I-ORDINAL', 'I-PRODUCT', 'I-LAW', 'I-LANGUAGE', 'L-ORG', 'L-DATE', 'L-PERSON', 'L-GPE', 'L-MONEY', 'L-CARDINAL', 'L-NORP', 'L-PERCENT', 'L-WORK_OF_ART', 'L-LOC', 'L-TIME', 'L-QUANTITY', 'L-FAC', 'L-EVENT', 'L-ORDINAL', 'L-PRODUCT', 'L-LAW', 'L-LANGUAGE', 'U-ORG', 'U-DATE', 'U-PERSON', 'U-GPE', 'U-MONEY', 'U-CARDINAL', 'U-NORP', 'U-PERCENT', 'U-WORK_OF_ART', 'U-LOC', 'U-TIME', 'U-QUANTITY', 'U-FAC', 'U-EVENT', 'U-ORDINAL', 'U-PRODUCT', 'U-LAW', 'U-LANGUAGE', 'O', 'B-TEAM', 'I-TEAM', 'L-TEAM', 'U-TEAM', 'B-PLAYER', 'I-PLAYER', 'L-PLAYER', 'U-PLAYER']


['tagger', 'parser']

In [0]:
from IPython.display import clear_output
from spacy.util import compounding , minibatch
n_iter = 150
optimizer = nlp.resume_training()
with nlp.disable_pipes(*other_pipes):  # only train NER
  sizes = compounding(1.0, 4.0, 1.001)
  last_loss = 42327.484182610715 
  for itn in tqdm(range(n_iter)):
      shuffle(TRAINING_DATA)
      batches = minibatch(TRAINING_DATA, size=sizes)
      losses = {}
      for batch in batches:
          texts, annotations = zip(*batch)
          nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
      clear_output(wait=True)
      test_model()
      print("Loss ", losses)
      print("Rate ", (last_loss - losses["ner"])/last_loss)
      last_loss = losses["ner"]








 33%|███▎      | 49/150 [44:54<1:29:16, 53.04s/it]


TESTING MODEL

	Smeltz leaves Phoenix for Indonesian club
		Entities [('Smeltz', 'PLAYER')]
		Tokens [('Smeltz', 'PLAYER', 3), ('leaves', '', 2), ('Phoenix', '', 2), ('for', '', 2), ('Indonesian', '', 2), ('club', '', 2)]

	Giroud may not get Chelsea exit in January as Inter consider summer move on a free
		Entities []
		Tokens [('Giroud', '', 2), ('may', '', 2), ('not', '', 2), ('get', '', 2), ('Chelsea', '', 2), ('exit', '', 2), ('in', '', 2), ('January', '', 2), ('as', '', 2), ('Inter', '', 2), ('consider', '', 2), ('summer', '', 2), ('move', '', 2), ('on', '', 2), ('a', '', 2), ('free', '', 2)]

	Oxlade-Chamberlain signs new four-year Liverpool contract
		Entities [('Oxlade-Chamberlain', 'PLAYER')]
		Tokens [('Oxlade', 'PLAYER', 3), ('-', 'PLAYER', 1), ('Chamberlain', 'PLAYER', 1), ('signs', '', 2), ('new', '', 2), ('four', '', 2), ('-', '', 2), ('year', '', 2), ('Liverpool', '', 2), ('contract', '', 2)]

	Gordon signs Celtic renewal after collapsed Chelsea deal
		Entities []
		To

KeyboardInterrupt: ignored

In [0]:
!pip install pathlib
from pathlib import Path
new_model_name = "en_fnew_md"
output_dir = "./content/gdrive/My Drive/project stuff/en_fnew_md"
output_dir = Path(output_dir)
if not output_dir.exists():
    output_dir.mkdir()
nlp.meta["name"] = new_model_name  # rename model
nlp.to_disk(output_dir)
print("Saved model to", output_dir)

## Cleaning Report data

In [0]:
!unzip /content/gdrive/My\ Drive/project\ stuff/fmodel.zip 

In [0]:
nlp = spacy.load("en_fnew_md")

In [0]:
df = pd.read_csv("transfer_data.csv")

In [0]:
def cleantext(text):
  text=text.replace("\n"," ")
  text=text.replace("Other English editions","")
  text=text[:-193]
  text.strip()
  return(text)
df["content"] = df["content"].apply(cleantext);

In [0]:
import re
def datefix(x):
  match = re.search(r"\d+/\d+/\d+",x)
  if match == None:
    print(x)
  else:
    return(str(match.group()))
df["date"] = df["date"].apply(datefix)

In [0]:
def checkemph(x):
  doc = nlp(x.lower())
  for token in doc.ents:
    if token.label_ == "PERSON":
      return False
  return True
filtered_df = df[df["emphasis"].apply(checkemph)]

In [0]:
df = filtered_df
del filtered_df

In [0]:
df.columns

Index(['content', 'date', 'emphasis', 'subscript', 'subtitle', 'title',
       'topic', 'url'],
      dtype='object')

In [0]:
categories = dict()
for topics in df["topic"]:
  tl = topics.split("+")
  for topic in tl:
    doc = nlp(topic)
    for token in doc.ents:
      if token.label_ == ""
    try:
      categories[topic] = categories[topic] + 1;
    except:
      categories[topic] = 1;

categories

{' Transfers ': 12580,
 ' Tottenham Hotspur ': 687,
 ' Olympique Lyonnais ': 212,
 ' Moussa Dembélé ': 31,
 ' Premier League ': 7097,
 ' Harry Kane ': 93,
 ' Ligue 1 ': 1602,
 ' Primera División ': 3228,
 ' Tello ': 1,
 ' Real Betis ': 71,
 ' Barcelona ': 1764,
 ' Fiorentina ': 69,
 ' Manchester City ': 982,
 ' Nicolás Otamendi ': 44,
 ' Burton Albion ': 3,
 ' Burton Albion v Manchester City ': 2,
 ' League Cup ': 80,
 ' Swansea City ': 79,
 ' Everton ': 382,
 ' Gylfi Sigurðsson ': 8,
 ' Swansea City v West Bromwich Albion ': 2,
 ' Neymar ': 472,
 ' PSG ': 1114,
 ' Chelsea ': 1757,
 ' Championship ': 277,
 ' Burnley ': 83,
 ' West Bromwich Albion ': 73,
 ' Charlton Athletic ': 6,
 ' Ike Ugbo ': 2,
 ' Lewis Baker ': 9,
 ' Conor Gallagher ': 2,
 ' Roda JC ': 2,
 ' Norwich City ': 31,
 ' Queens Park Rangers ': 7,
 ' Crystal Palace ': 210,
 ' Hull City ': 30,
 ' Roma ': 328,
 ' Manchester United ': 2164,
 ' Serie A ': 2144,
 ' Chris Smalling ': 15,
 ' Borussia Dortmund ': 481,
 ' Erling Hå

In [0]:
doc = nlp("there is a huge hole going to be left behind by Lionel Messi when he leaves Barcelona")
for token in doc.ents:
  print(token.text,token.label_)

there TEAM
Lionel Messi PLAYER
Barcelona TEAM
